# TP 01 - Le Mécanisme d'Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre et implémenter le mécanisme d'attention, brique fondamentale des Transformers

---

## Objectifs pédagogiques

À la fin de ce TP, vous serez capable de :
1. Expliquer intuitivement ce qu'est l'attention
2. Comprendre les concepts de Query, Key, Value
3. Implémenter le Scaled Dot-Product Attention
4. Visualiser et interpréter les poids d'attention

## 0. Installation et imports

Exécutez cette cellule pour installer les dépendances nécessaires.

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

---

## 1. Introduction : Pourquoi l'attention ?

### Le problème des RNN

Les réseaux récurrents (RNN, LSTM) traitent les séquences **mot par mot**, ce qui pose deux problèmes :
1. **Dépendances longues** : difficile de relier des mots éloignés
2. **Pas de parallélisation** : calcul séquentiel = lent

### L'idée de l'attention

L'attention permet à chaque élément d'une séquence de "regarder" tous les autres éléments et de décider lesquels sont importants.

**Exemple** : Dans la phrase *"Le chat qui était sur le toit a sauté"*, pour comprendre *"a sauté"*, le modèle doit se concentrer sur *"chat"* (le sujet) plutôt que sur *"toit"*.

### Analogie : La recherche dans une base de données

Imaginez une bibliothèque :
- **Query (Q)** : Votre question ("Je cherche un livre sur les chats")
- **Key (K)** : Les mots-clés de chaque livre
- **Value (V)** : Le contenu des livres

L'attention compare votre **question** aux **mots-clés**, puis retourne un mélange pondéré des **contenus** les plus pertinents.

---

## 2. Visualisation intuitive

Avant de coder, visualisons ce que fait l'attention.

In [ ]:
# Exemple simple : attention dans une phrase
phrase = ["Le", "chat", "mange", "la", "souris"]

# Matrice d'attention simulée (quels mots regardent quels mots ?)
# Chaque ligne = un mot qui "regarde" les autres
attention_simulee = torch.tensor([
    [0.8, 0.1, 0.05, 0.03, 0.02],  # "Le" regarde surtout lui-même
    [0.1, 0.7, 0.1, 0.05, 0.05],   # "chat" regarde surtout lui-même
    [0.05, 0.4, 0.4, 0.05, 0.1],   # "mange" regarde "chat" et lui-même
    [0.02, 0.03, 0.05, 0.8, 0.1],  # "la" regarde surtout lui-même
    [0.02, 0.1, 0.2, 0.08, 0.6],   # "souris" regarde "mange" et elle-même
])

# Visualisation
plt.figure(figsize=(8, 6))
plt.imshow(attention_simulee, cmap='Blues')
plt.xticks(range(5), phrase)
plt.yticks(range(5), phrase)
plt.xlabel("Mots regardés (Keys)")
plt.ylabel("Mots qui regardent (Queries)")
plt.title("Qui regarde qui ? (Matrice d'attention)")
plt.colorbar(label="Poids d'attention")

# Afficher les valeurs
for i in range(5):
    for j in range(5):
        plt.text(j, i, f'{attention_simulee[i,j]:.2f}', 
                ha='center', va='center',
                color='white' if attention_simulee[i,j] > 0.5 else 'black')
plt.show()

**Question** : Dans cette matrice, quel mot le verbe "mange" regarde-t-il le plus ? Pourquoi est-ce logique ?

---

## 3. Scaled Dot-Product Attention

### La formule

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Où :
- $Q$ (Query) : Ce que je cherche - shape `(seq_len, d_k)`
- $K$ (Key) : Les étiquettes de ce qui est disponible - shape `(seq_len, d_k)`
- $V$ (Value) : Le contenu disponible - shape `(seq_len, d_v)`
- $d_k$ : Dimension des clés (pour normaliser)

### Décomposition étape par étape

1. **Scores** : $QK^T$ - Mesure la similarité entre queries et keys
2. **Scaling** : Division par $\sqrt{d_k}$ - Évite des valeurs trop grandes
3. **Softmax** : Transforme en probabilités (somme = 1)
4. **Output** : Multiplication par $V$ - Moyenne pondérée des values

### Exercice 1 : Calcul manuel des scores

Commençons par calculer les scores d'attention manuellement.

In [ ]:
# Exemple simple avec 3 mots et dimension 4
seq_len = 3
d_k = 4

# Créons des Query, Key, Value aléatoires
Q = torch.randn(seq_len, d_k)
K = torch.randn(seq_len, d_k)
V = torch.randn(seq_len, d_k)

print("Q (Queries):")
print(Q)
print(f"\nShape Q: {Q.shape}")
print(f"Shape K: {K.shape}")
print(f"Shape V: {V.shape}")

In [ ]:
# ============================================
# EXERCICE 1 : Calculez les scores d'attention
# ============================================

# Étape 1 : Calculer Q @ K^T (produit matriciel)
# La transposée de K se note K.T ou K.transpose(-2, -1)

scores = None  # TODO: Calculer Q @ K^T

print("Scores (Q @ K^T):")
print(scores)
print(f"Shape: {scores.shape}")  # Devrait être (3, 3)

In [ ]:
# ============================================
# EXERCICE 2 : Appliquez le scaling
# ============================================

# Diviser par sqrt(d_k) pour éviter des valeurs trop grandes
# Indice : utilisez d_k ** 0.5 ou torch.sqrt(torch.tensor(d_k))

import math

scaled_scores = None  # TODO: scores / sqrt(d_k)

print("Scaled scores:")
print(scaled_scores)

In [ ]:
# ============================================
# EXERCICE 3 : Appliquez le softmax
# ============================================

# Le softmax transforme les scores en probabilités
# Chaque ligne doit sommer à 1
# Indice : F.softmax(tensor, dim=-1) applique softmax sur la dernière dimension

attention_weights = None  # TODO: Appliquer softmax sur scaled_scores

print("Poids d'attention (après softmax):")
print(attention_weights)
print(f"\nVérification - Somme par ligne: {attention_weights.sum(dim=-1)}")

In [ ]:
# ============================================
# EXERCICE 4 : Calculez la sortie finale
# ============================================

# Multiplier les poids d'attention par V
# C'est une moyenne pondérée des values

output = None  # TODO: attention_weights @ V

print("Output:")
print(output)
print(f"Shape: {output.shape}")  # Devrait être (3, 4)

---

## 4. Implémentation complète

### Exercice 5 : Fonction d'attention

Maintenant, regroupez tout dans une fonction.

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (seq_len, d_k) ou (batch, seq_len, d_k)
        K: Keys, shape (seq_len, d_k) ou (batch, seq_len, d_k)
        V: Values, shape (seq_len, d_v) ou (batch, seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention, shape (seq_len, d_v)
        attention_weights: Poids d'attention, shape (seq_len, seq_len)
    """
    # Récupérer d_k (dernière dimension de K)
    d_k = K.shape[-1]
    
    # TODO: Implémenter les 4 étapes
    # 1. Calculer les scores : Q @ K^T
    scores = None
    
    # 2. Scaling : diviser par sqrt(d_k)
    scaled_scores = None
    
    # 3. Softmax pour obtenir les poids
    attention_weights = None
    
    # 4. Moyenne pondérée : weights @ V
    output = None
    
    return output, attention_weights

In [ ]:
# Test de votre fonction
Q_test = torch.randn(4, 8)  # 4 tokens, dimension 8
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Output shape: {output.shape}")  # Attendu: (4, 8)
print(f"Weights shape: {weights.shape}")  # Attendu: (4, 4)
print(f"Weights sum per row: {weights.sum(dim=-1)}")  # Attendu: [1, 1, 1, 1]

---

## 5. Pourquoi diviser par sqrt(d_k) ?

C'est une question importante ! Voyons l'effet du scaling.

In [ ]:
# Comparaison avec et sans scaling
d_k_grand = 512  # Dimension typique dans un Transformer

Q_grand = torch.randn(10, d_k_grand)
K_grand = torch.randn(10, d_k_grand)

# Scores sans scaling
scores_sans_scaling = Q_grand @ K_grand.T
attention_sans_scaling = F.softmax(scores_sans_scaling, dim=-1)

# Scores avec scaling
scores_avec_scaling = (Q_grand @ K_grand.T) / math.sqrt(d_k_grand)
attention_avec_scaling = F.softmax(scores_avec_scaling, dim=-1)

print("=== SANS SCALING ===")
print(f"Scores - min: {scores_sans_scaling.min():.2f}, max: {scores_sans_scaling.max():.2f}")
print(f"Attention max par ligne: {attention_sans_scaling.max(dim=-1).values[:3]}")
print(f"Entropie moyenne: {-(attention_sans_scaling * attention_sans_scaling.log()).sum(dim=-1).mean():.4f}")

print("\n=== AVEC SCALING ===")
print(f"Scores - min: {scores_avec_scaling.min():.2f}, max: {scores_avec_scaling.max():.2f}")
print(f"Attention max par ligne: {attention_avec_scaling.max(dim=-1).values[:3]}")
print(f"Entropie moyenne: {-(attention_avec_scaling * attention_avec_scaling.log()).sum(dim=-1).mean():.4f}")

**Observation** : Sans scaling, le softmax devient très "peaked" (une valeur proche de 1, les autres proches de 0). Le scaling permet une distribution plus douce et des gradients plus stables.

---

## 6. Application : Self-Attention sur une phrase

Appliquons l'attention à une vraie phrase et visualisons les résultats.

In [ ]:
# Phrase exemple
phrase = ["Le", "chat", "noir", "dort", "sur", "le", "canapé"]
seq_len = len(phrase)
embed_dim = 16  # Dimension des embeddings

# Simulons des embeddings (en vrai, ils seraient appris)
torch.manual_seed(42)
embeddings = torch.randn(seq_len, embed_dim)

print(f"Phrase: {phrase}")
print(f"Embeddings shape: {embeddings.shape}")

In [ ]:
# En self-attention, Q = K = V = embeddings
# (chaque mot se compare à tous les autres)

output, attention_weights = scaled_dot_product_attention(
    Q=embeddings,
    K=embeddings,
    V=embeddings
)

# Visualisation
plt.figure(figsize=(10, 8))
plt.imshow(attention_weights.detach().numpy(), cmap='Blues')
plt.xticks(range(seq_len), phrase, rotation=45)
plt.yticks(range(seq_len), phrase)
plt.xlabel("Mots regardés (Keys)")
plt.ylabel("Mots qui regardent (Queries)")
plt.title("Self-Attention : Qui regarde qui ?")
plt.colorbar(label="Poids d'attention")

# Afficher les valeurs
for i in range(seq_len):
    for j in range(seq_len):
        val = attention_weights[i, j].item()
        plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                color='white' if val > 0.3 else 'black', fontsize=8)
plt.tight_layout()
plt.show()

---

## 7. Module nn.Module

### Exercice 6 : Classe Attention en PyTorch

Créons une classe PyTorch réutilisable.

In [ ]:
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    En self-attention, on projette les embeddings en Q, K, V
    avec des matrices de poids apprenables.
    """
    
    def __init__(self, embed_dim):
        """
        Args:
            embed_dim: Dimension des embeddings d'entrée
        """
        super().__init__()
        self.embed_dim = embed_dim
        
        # TODO: Créer 3 couches linéaires pour projeter vers Q, K, V
        # Chaque couche : embed_dim -> embed_dim
        self.W_q = None  # nn.Linear(embed_dim, embed_dim)
        self.W_k = None  # nn.Linear(embed_dim, embed_dim)
        self.W_v = None  # nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)
        
        Returns:
            output: Résultat de l'attention
            attention_weights: Poids d'attention
        """
        # TODO: Projeter x vers Q, K, V
        Q = None  # self.W_q(x)
        K = None  # self.W_k(x)
        V = None  # self.W_v(x)
        
        # TODO: Appliquer l'attention
        # Attention: pour les batches, K.transpose(-2, -1) au lieu de K.T
        d_k = self.embed_dim
        
        scores = None  # Q @ K.transpose(-2, -1)
        scaled_scores = None  # scores / sqrt(d_k)
        attention_weights = None  # softmax
        output = None  # attention_weights @ V
        
        return output, attention_weights

In [ ]:
# Test du module
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # Attendu: (2, 5, 32)
print(f"Weights shape: {weights.shape}")  # Attendu: (2, 5, 5)

---

## 8. Exercice de synthèse

Analysez les poids d'attention appris sur un exemple concret.

In [ ]:
# Phrase de test
phrase_test = ["Le", "programmeur", "écrit", "du", "code", "Python"]
seq_len = len(phrase_test)

# Embeddings simulés
torch.manual_seed(123)
x = torch.randn(1, seq_len, embed_dim)  # Batch size = 1

# Appliquer l'attention
attention_layer = SelfAttention(embed_dim)
output, weights = attention_layer(x)

# Visualiser
plt.figure(figsize=(10, 8))
plt.imshow(weights[0].detach().numpy(), cmap='Blues')
plt.xticks(range(seq_len), phrase_test, rotation=45)
plt.yticks(range(seq_len), phrase_test)
plt.xlabel("Keys")
plt.ylabel("Queries")
plt.title("Self-Attention (poids aléatoires non entraînés)")
plt.colorbar()

for i in range(seq_len):
    for j in range(seq_len):
        val = weights[0, i, j].item()
        plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                color='white' if val > 0.3 else 'black', fontsize=9)
plt.tight_layout()
plt.show()

---

## 9. Récapitulatif

### Ce que nous avons appris

1. **L'attention** permet à chaque élément de "regarder" tous les autres
2. **Q, K, V** : Query (ce que je cherche), Key (les étiquettes), Value (le contenu)
3. **Formule** : $\text{softmax}(QK^T / \sqrt{d_k}) \cdot V$
4. **Scaling** : Essentiel pour la stabilité des gradients

### Points clés

| Concept | Rôle |
|---------|------|
| Dot product $QK^T$ | Mesure la similarité |
| Softmax | Transforme en probabilités |
| Scaling $\sqrt{d_k}$ | Stabilise les gradients |
| Self-attention | Q = K = V (chaque mot regarde tous les autres) |

### Prochaine session

Nous verrons le **Multi-Head Attention** : plusieurs "têtes" d'attention qui regardent sous différents angles.

---

## 10. Pour aller plus loin (optionnel)

### Exercice bonus : Masking

Dans certains cas (génération de texte), on veut empêcher un mot de "voir" les mots futurs.

In [ ]:
def scaled_dot_product_attention_with_mask(Q, K, V, mask=None):
    """
    Attention avec masking optionnel.
    
    Args:
        Q, K, V: Query, Key, Value
        mask: Tensor booléen, True = position à masquer
    """
    d_k = K.shape[-1]
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)
    
    # Appliquer le masque (mettre -inf pour les positions masquées)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    attention_weights = F.softmax(scores, dim=-1)
    output = attention_weights @ V
    
    return output, attention_weights

# Créer un masque causal (triangulaire)
seq_len = 5
causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
print("Masque causal (True = masqué):")
print(causal_mask.int())

In [ ]:
# Test avec masque
Q = torch.randn(seq_len, 8)
K = torch.randn(seq_len, 8)
V = torch.randn(seq_len, 8)

output_masked, weights_masked = scaled_dot_product_attention_with_mask(Q, K, V, causal_mask)

print("Poids d'attention avec masque causal:")
print(weights_masked.round(decimals=2))
print("\nObservation: chaque ligne ne peut voir que les positions précédentes (et elle-même)")